In [2]:
import numpy as np
import json
import time
import sys
import redis
import msgpack
from multiprocessing import Process, Manager

In [3]:
# Random array of floats sized 1858
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
#r = redis.Redis(connection_pool=pool)
fake_policy = np.random.rand(1858).astype(np.float32)
fake_value = np.random.rand(1).astype(np.float32)
fake_policy, fake_value

(array([0.56664246, 0.5617776 , 0.9995359 , ..., 0.7664546 , 0.9325946 ,
        0.69126344], dtype=float32),
 array([0.91439945], dtype=float32))

In [4]:
times = []
for i in range(100):
    t1 = time.time()
    fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
    json_str = json.dumps({
        "policy": fake_policy_quantized.tolist(),
        "value": fake_value.tolist()
    })
    r.set("test", json_str)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to serialize: ", np.mean(times))

Average time to serialize:  0.00033260107040405273


In [4]:
times = []
for i in range(100):
    t1 = time.time()
    json_redis = r.get("test")
    data = json.loads(json_redis)
    fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
    fake_value_out = np.array(data["value"], dtype=np.float32)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to deserialize: ", np.mean(times))

Average time to deserialize:  0.0002930087804794312


In [5]:
sys.getsizeof(fake_policy), sys.getsizeof(json_str), sys.getsizeof(json_redis), sys.getsizeof(data), sys.getsizeof(fake_policy_out)

(7544, 8600, 8584, 232, 7544)

In [26]:
times = []
for i in range(100):
    t1 = time.time()
    fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
    package = msgpack.dumps({
        "policy": fake_policy.tolist(),
        "value": fake_value.tolist()
    })
    r.set("test2", package)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to serialize: ", np.mean(times))

Average time to serialize:  0.00032272100448608396


In [27]:
times = []
for i in range(100):
    t1 = time.time()
    package_redis = r.get("test2")
    data = msgpack.loads(package_redis)
    fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
    fake_value_out = np.array(data["value"], dtype=np.float32)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to deserialize: ", np.mean(times))

Average time to deserialize:  0.00036874055862426755


In [25]:
sys.getsizeof(fake_policy), sys.getsizeof(package), sys.getsizeof(package_redis), sys.getsizeof(data), sys.getsizeof(fake_policy_out)

(7544, 16782, 2856, 232, 7544)

In [11]:
fake_policy_out, fake_value_out

(array([0.9254902 , 0.13333334, 0.9137255 , ..., 0.84313726, 0.1764706 ,
        0.7764706 ], dtype=float32),
 array([0.44071597], dtype=float32))

In [15]:

def test_write(cache, pool):
    r = redis.Redis(connection_pool=pool)
    times = []
    for i in range(100):
        t1 = time.time()
        fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
        json_str = json.dumps({
            "policy": fake_policy_quantized.tolist(),
            "value": fake_value.tolist()
        })
        r.set("test", json_str)
        t2 = time.time()
        times.append(t2 - t1)
    print("Average time to serialize: ", np.mean(times))

def test_read(cache, pool):
    r = redis.Redis(connection_pool=pool)
    times = []
    for i in range(100):
        t1 = time.time()
        package_redis = r.get("test2")
        data = msgpack.loads(package_redis)
        fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
        fake_value_out = np.array(data["value"], dtype=np.float32)
        t2 = time.time()
        times.append(t2 - t1)
    print("Average time to deserialize: ", np.mean(times))

with Manager() as manager:
    cache = manager.dict()
    p1 = Process(target=test_write, args=(cache, pool))
    p2 = Process(target=test_read, args=(cache, pool))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(len(cache), sys.getsizeof(cache))

Average time to serialize: Average time to deserialize:   0.000284445285797119130.00037267684936523435

0 48


In [19]:
r = redis.Redis(connection_pool=pool)
times = []
for i in range(100):
    t1 = time.time()
    n = r.get("test6")
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to fetch None: ", np.mean(times))

Average time to fetch None:  7.092714309692383e-05


In [20]:
np.mean(times) * 1000000

70.92714309692383

In [7]:
from rchess import Board
import timeit

def time_legal_moves():
    board = Board()
    for i in range(10):
        legal_moves = board.legal_moves()
    return board

def time_legal_moves_num():
    board = Board()
    for i in range(10):
        legal_moves_num = board.legal_moves_num()
    return board

def time_legal_moves_uci():
    board = Board()
    for i in range(10):
        legal_moves_uci = board.legal_moves_uci()
    return board

# time all three and print the results
legal_moves = timeit.timeit(time_legal_moves, number=10000)
legal_moves_num = timeit.timeit(time_legal_moves_num, number=10000)
legal_moves_uci = timeit.timeit(time_legal_moves_uci, number=10000)

print(f"legal_moves: {legal_moves}")
print(f"legal_moves_num: {legal_moves_num}")
print(f"legal_moves_uci: {legal_moves_uci}")


legal_moves: 0.10827130500001658
legal_moves_num: 0.04818951000015659
legal_moves_uci: 0.2719182629998613


In [3]:
from rchess import Board
import timeit

def time_legal_moves():
    board = Board()
    for i in range(10):
        legal_moves = board.legal_moves()
        uci = legal_moves[0].uci()
    return board

def time_legal_moves_num():
    board = Board()
    for i in range(10):
        legal_moves_num = board.legal_moves_num()
    return board

def time_legal_moves_uci():
    board = Board()
    for i in range(10):
        legal_moves_uci = board.legal_moves_uci()
    return board

def time_legal_moves_tuple():
    board = Board()
    for i in range(10):
        legal_moves_uci = board.legal_moves_tuple()
    return board

# time all three and print the results
legal_moves = timeit.timeit(time_legal_moves, number=10000)
legal_moves_num = timeit.timeit(time_legal_moves_num, number=10000)
legal_moves_uci = timeit.timeit(time_legal_moves_uci, number=10000)
legal_moves_tuple = timeit.timeit(time_legal_moves_tuple, number=10000)

print(f"legal_moves: {legal_moves}")
print(f"legal_moves_num: {legal_moves_num}")
print(f"legal_moves_uci: {legal_moves_uci}")
print(f"legal_moves_tuple: {legal_moves_tuple}")

legal_moves: 0.11833564499988825
legal_moves_num: 0.04872912600012569
legal_moves_uci: 0.10376740599986078
legal_moves_tuple: 0.149593976999995
